# LSTM ile CPU Kullanımı Tahmin Modeli

In [29]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import time
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Adım 1: Kurulum ve Cihaz Belirleme

In [30]:
# Tüm hesaplamaların yapılacağı cihazı belirle (GPU varsa GPU, yoksa CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Hesaplamalar için kullanılacak cihaz: {device}')

Hesaplamalar için kullanılacak cihaz: cuda


## Adım 2: Veri Yükleme ve Ön İşleme

In [31]:
# İşlenmiş veriyi oku
df = pd.read_csv('genel_sunucu_yuku_tum_metrikler.csv', index_col='timestamp')
df.index = pd.to_datetime(df.index, unit='s')

# Tahmin için 3 sütunu da kullan
ts_data = df[['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']].values

# Veriyi 0-1 arasına ölçeklendir
scaler = MinMaxScaler(feature_range=(0, 1))
ts_data_scaled = scaler.fit_transform(ts_data)

# Veriyi eğitim ve test setlerine ayır
train_size = int(len(ts_data_scaled) * 0.8)
train_data, test_data = ts_data_scaled[:train_size], ts_data_scaled[train_size:]

print("Veri yüklendi, ölçeklendi ve bölündü.")

Veri yüklendi, ölçeklendi ve bölündü.


## Adım 3: Zaman Serisi Dizileri Oluşturma

In [32]:
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window):
        feature = data[i:(i + lookback_window)]
        target = data[i + lookback_window]
        X.append(feature)
        y.append(target)
    # Bu aşamada tensörleri CPU'da oluşturuyoruz, eğitim sırasında GPU'ya göndereceğiz
    return torch.FloatTensor(X), torch.FloatTensor(y)

lookback = 12  # 1 saatlik veri

X_train, y_train = create_sequences(train_data, lookback)
X_test, y_test = create_sequences(test_data, lookback)

print("Eğitim verisi şekli (X, y):", X_train.shape, y_train.shape)
print("Test verisi şekli (X, y):", X_test.shape, y_test.shape)

Eğitim verisi şekli (X, y): torch.Size([6899, 12, 3]) torch.Size([6899, 3])
Test verisi şekli (X, y): torch.Size([1716, 12, 3]) torch.Size([1716, 3])


## Adım 4: LSTM Modelini Tanımlama ve GPU'ya Taşıma

In [33]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_layer_size=50, output_size=3):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq)
        predictions = self.linear(lstm_out[:, -1, :])
        return predictions

# Modeli oluştur
model = LSTMModel()
# Modeli doğrudan GPU'ya taşı
model.to(device)

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Model Mimarisi:")
print(model)

Model Mimarisi:
LSTMModel(
  (lstm): LSTM(3, 50, batch_first=True)
  (linear): Linear(in_features=50, out_features=3, bias=True)
)


## Adım 5: Modeli GPU Üzerinde Eğitme

In [34]:
epochs = 15

print(f"Model {epochs} epoch boyunca '{device}' üzerinde eğitiliyor...")
start_time = time.time()
model.train()

for i in range(epochs):
    for seq, labels in zip(X_train, y_train):
        optimizer.zero_grad()
        
        # Veriyi her adımda GPU'ya gönder
        seq_gpu = seq.to(device)
        labels_gpu = labels.to(device)
        
        # Modeli çalıştır
        y_pred = model(seq_gpu.unsqueeze(0))
        
        # Loss'u hesapla ve geri yayılım yap
        single_loss = loss_function(y_pred.squeeze(0), labels_gpu)
        single_loss.backward()
        optimizer.step()


    print(f'Epoch: {i+1:3} loss: {single_loss.item():10.8f}')

end_time = time.time()
print(f"Eğitim tamamlandı. Süre: {end_time - start_time:.2f} saniye")

Model 15 epoch boyunca 'cuda' üzerinde eğitiliyor...
Epoch:   1 loss: 0.00089344
Epoch:   2 loss: 0.00082845
Epoch:   3 loss: 0.00076230
Epoch:   4 loss: 0.00049219
Epoch:   5 loss: 0.00058004
Epoch:   6 loss: 0.00064951
Epoch:   7 loss: 0.00056523
Epoch:   8 loss: 0.00053856
Epoch:   9 loss: 0.00057531
Epoch:  10 loss: 0.00080738
Epoch:  11 loss: 0.00072828
Epoch:  12 loss: 0.00075520
Epoch:  13 loss: 0.00071042
Epoch:  14 loss: 0.00064727
Epoch:  15 loss: 0.00061591
Eğitim tamamlandı. Süre: 85.11 saniye


## Adım 6: GPU ile Tahmin Yapma

In [35]:
model.eval()
print("Test verisi üzerinde tahminler yapılıyor...")

test_predictions = []
with torch.no_grad():
    for seq, labels in zip(X_test, y_test):
        # Tahmin için veriyi GPU'ya gönder
        seq_gpu = seq.to(device)
        
        y_pred = model(seq_gpu.unsqueeze(0))
        
        # Sonucu CPU'ya geri alıp numpy'a çeviriyoruz
        test_predictions.append(y_pred.squeeze(0).cpu().numpy())

print("Tahminler tamamlandı.")

Test verisi üzerinde tahminler yapılıyor...
Tahminler tamamlandı.


## Adım 7: Sonuçları Değerlendirme ve Görselleştirme

In [36]:
# Tahminleri ve gerçek değerleri orijinal ölçeğine geri çevir
predictions_array = np.array(test_predictions)
predictions_original_scale = scaler.inverse_transform(predictions_array)
y_test_original_scale = scaler.inverse_transform(y_test.numpy())


# MAPE için özel fonksiyonumuzu tanımlayalım
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100

target_columns = ['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']

print("--- LSTM Model Performans Metrikleri (GPU) ---")
for i, column in enumerate(target_columns):
    y_true_col = y_test_original_scale[:, i]
    y_pred_col = predictions_original_scale[:, i]
    
    # Tüm metrikleri hesapla
    mae = mean_absolute_error(y_true_col, y_pred_col)
    rmse = np.sqrt(mean_squared_error(y_true_col, y_pred_col))
    r2 = r2_score(y_true_col, y_pred_col)
    mape = mean_absolute_percentage_error(y_true_col, y_pred_col)
    
    print(f"\n--- Metrik: '{column}' ---")
    print(f"  Ortalama Mutlak Hata (MAE): {mae:.2f}")
    print(f"  Kök Ortalama Kare Hata (RMSE): {rmse:.2f}")
    print(f"  R-Kare (R²): {r2:.2f}")
    print(f"  Ortalama Mutlak Yüzde Hata (MAPE): {mape:.2f}%")

--- LSTM Model Performans Metrikleri (GPU) ---

--- Metrik: 'avg_min_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.10
  Kök Ortalama Kare Hata (RMSE): 0.12
  R-Kare (R²): 0.69
  Ortalama Mutlak Yüzde Hata (MAPE): 1.59%

--- Metrik: 'avg_max_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.17
  Kök Ortalama Kare Hata (RMSE): 0.24
  R-Kare (R²): 0.83
  Ortalama Mutlak Yüzde Hata (MAPE): 0.87%

--- Metrik: 'avg_avg_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.12
  Kök Ortalama Kare Hata (RMSE): 0.15
  R-Kare (R²): 0.82
  Ortalama Mutlak Yüzde Hata (MAPE): 1.19%


In [37]:
# Sonuçları görselleştir
fig = go.Figure()

for i, column in enumerate(target_columns):
    fig.add_trace(go.Scatter(
        x=df.index[train_size+lookback:],
        y=y_test_original_scale[:, i],
        mode='lines',
        name=f'Gerçek {column}'
    ))
    fig.add_trace(go.Scatter(
        x=df.index[train_size+lookback:],
        y=predictions_original_scale[:, i],
        mode='lines',
        name=f'Tahmin {column}',
        line=dict(dash='dash')
    ))

fig.update_layout(
    title='CPU Metrikleri: Gerçek Değerler vs. LSTM Tahminleri (GPU)',
    xaxis_title='Zaman',
    yaxis_title='Ortalama CPU Yükü (%)'
)

fig.show()

In [38]:
print("Tüm metrikler için birleştirilmiş sonuçlar görselleştiriliyor...")

# Grafiği oluştur
fig = go.Figure()

# Tahmin ettiğimiz sütunların isimleri
target_columns = ['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']

# Her bir metrik için gerçek ve tahmin değerlerini aynı grafiğe çiz
# enumerate kullanarak hem sütun ismine hem de index'e (0, 1, 2) ulaşıyoruz
for i, column in enumerate(target_columns):
    # Gerçek Değerler (Test)
    fig.add_trace(go.Scatter(
        x=df.index[train_size+lookback:],
        y=y_test_original_scale[:, i], # Numpy dizisinden i'ninci sütunu al
        mode='lines',
        name=f'Gerçek {column}'
    ))
    
    # Model Tahminleri
    fig.add_trace(go.Scatter(
        x=df.index[train_size+lookback:],
        y=predictions_original_scale[:, i], # Numpy dizisinden i'ninci sütunu al
        mode='lines',
        name=f'Tahmin {column}',
        line=dict(dash='dash') # Tahmin çizgilerini kesikli yap
    ))

fig.update_layout(
    title='CPU Metrikleri: Gerçek Değerler vs. LSTM Tahminleri (Hepsi Bir Arada)',
    xaxis_title='Zaman',
    yaxis_title='Ortalama CPU Yükü (%)'
)

fig.show()

Tüm metrikler için birleştirilmiş sonuçlar görselleştiriliyor...


In [39]:
print("Sonuçlar ayri görselleştiriliyor...")

# Her bir metrik için ayrı ayrı grafikler oluşturalım
for i, col_name in enumerate(['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']):
    fig = go.Figure()

    # Gerçek Test Verisi
    fig.add_trace(go.Scatter(
        x=df.index[train_size+lookback:],
        y=y_test_original_scale[:, i], # Sadece i'ninci sütunu al
        mode='lines',
        name='Gerçek Değerler (Test)'
    ))

    # Model Tahminleri
    fig.add_trace(go.Scatter(
        x=df.index[train_size+lookback:],
        y=predictions_original_scale[:, i], # Sadece i'ninci sütunu al
        mode='lines',
        name='LSTM Tahminleri'
    ))

    fig.update_layout(
        title=f'{col_name} için Gerçek Değerler vs. LSTM Tahminleri',
        xaxis_title='Zaman',
        yaxis_title='Ortalama CPU Yükü (%)'
    )

    fig.show()


Sonuçlar ayri görselleştiriliyor...
